# NLP beginer

https://github.com/proger-n/DS_01_Tweets/blob/main/src/tweets.ipynb

### Introduction
NLP (natural language processing) — это область знаний и ряд методов и алгоритмов, которые помогают обрабатывать текстовые данные и извлекать из них информацию. Вы уже работали со структурированными данными — таблицами, полными непрерывных и категориальных признаков. Текст является примером полуструктурированных данных. С ним нужно что-то делать, чтобы использовать его, например, в задачах машинного обучения.

ЦЕЛЬ РАБОТЫ - предварительная обработка тектовых данных и обучение с помощью различных классификаторов для решения задачи классификации твитов на три класса (негативный, отрицательный и нейтральный)

### 1. Предварительная обработка текста:
Здесь перед подачей в классификатор необходимо предобработать текст, есть следующие способы:
- [Term Frequency-Inverse Document Frequency (TF-IDF)](https://habr.com/ru/companies/otus/articles/755772/) - это один из наиболее распространенных и мощных методов для извлечения признаков из текстовых данных. TF-IDF вычисляет важность каждого слова в документе относительно количества его употреблений в данном документе и во всей коллекции текстов. Этот метод позволяет выделить ключевые слова и понять, какие слова имеют больший вес для определенного документа в контексте всей коллекции.
Перед тем как вычислять TF-IDF, мы должны выполнить предварительную обработку, такую как удаление стоп-слов, приведение к нижнему регистру и токенизация — разбиение текстов на отдельные слова или токены.
- Стемминг (stemming) - это грубый эвристический процесс, который отрезает «лишнее» от корня слов, часто это приводит к потере словообразовательных суффиксов. Например слово "Коты", станет "Кот"
- Лематизация - это более тонкий процесс, который использует словарь и морфологический анализ, чтобы в итоге привести слово к его канонической форме – лемме.

Примеры лематизации и стемминга: Слово good – это лемма для слова better. Стеммер не увидит эту связь, так как здесь нужно сверяться со словарем.
Слово play – это базовая форма слова playing. Тут справятся и стемминг, и лемматизация.
Слово meeting может быть как нормальной формой существительного, так и формой глагола to meet, в зависимости от контекста.
В отличие от стемминга, лемматизация попробует выбрать правильную лемму,опираясь на контекст.

## 0.Import

In [107]:
import pandas as pd
import numpy as np
import zipfile
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from autocorrect import Speller
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/nikolai/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

## 1. Подготовка данных
Данные хранятся в архиве. в Архиве три файла csv. В одном файле негативные отзыва, в другом позитивные и в третьем нейтральные. csv файлы содержат одну строку и много столбцов, поэтому нужно будет трансопнировать и привести всё в одну таблицу

In [2]:
# Распаковываем zip архив
with zipfile.ZipFile('../datasets/p00_tweets.zip', 'r') as zip_ref:
    zip_ref.extractall('data')
    
# Читаем файлы в pandas
positive_tweets = pd.read_csv('data/processedPositive.csv')
print(positive_tweets.shape)

neutral_tweets = pd.read_csv('data/processedNeutral.csv')
print(neutral_tweets.shape)

negative_tweets = pd.read_csv('data/processedNegative.csv')
print(negative_tweets.shape)

(0, 1186)
(0, 1570)
(0, 1117)


### 1.1. Приведем все данные в одну таблицу.
Для этого транспонируем исходные таблицы и назначим индексы
- 1 - негативный твит
- 2 - нейтральный твит
- 3 - положительный твит

In [3]:
def prepare_data(df, target_index):
    df_new = df.T.reset_index().rename(columns={'index': 'tweet'})
    df_new['target'] = target_index
    return df_new

In [4]:
negative_tweets = prepare_data(negative_tweets, target_index=1)
neutral_tweets = prepare_data(neutral_tweets, target_index=2)
positive_tweets = prepare_data(positive_tweets, target_index=3)

df = pd.concat([negative_tweets, neutral_tweets, positive_tweets], ignore_index=True)
df

,tweet,target
0,How unhappy some dogs like it though,1
1,talking to my over driver about where I'm goin...,1
2,Does anybody know if the Rand's likely to fall...,1
3,I miss going to gigs in Liverpool unhappy,1
4,There isnt a new Riverdale tonight ? unhappy,1
...,...,...
3868,Thanks for the recent follow Happy to connect ...,3
3869,- top engaged members this week happy,3
3870,ngam to weeks left for cadet pilot exam cryin...,3
3871,Great! You're welcome Josh happy ^Adam,3


### 1.2. чистим твиты
убираем все символы кроме букв и преобразуем к нижнему регистру

In [5]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r'[^a-zA-Z ]', '', regex=True)
    df[text_field] = df[text_field].str.lower()
    return df

df = standardize_text(df, 'tweet')

df

,tweet,target
0,how unhappy some dogs like it though,1
1,talking to my over driver about where im going...,1
2,does anybody know if the rands likely to fall ...,1
3,i miss going to gigs in liverpool unhappy,1
4,there isnt a new riverdale tonight unhappy,1
...,...,...
3868,thanks for the recent follow happy to connect ...,3
3869,top engaged members this week happy,3
3870,ngam to weeks left for cadet pilot exam cryin...,3
3871,great youre welcome josh happy adam,3


### 1.3. Токенизация
Разбиваем твиты на токены

In [7]:
tokenizer = RegexpTokenizer(r'\w+')

df['tokens'] = df['tweet'].apply(tokenizer.tokenize)
df.head()

,tweet,target,tokens
0,how unhappy some dogs like it though,1,"[how, unhappy, some, dogs, like, it, though]"
1,talking to my over driver about where im going...,1,"[talking, to, my, over, driver, about, where, ..."
2,does anybody know if the rands likely to fall ...,1,"[does, anybody, know, if, the, rands, likely, ..."
3,i miss going to gigs in liverpool unhappy,1,"[i, miss, going, to, gigs, in, liverpool, unha..."
4,there isnt a new riverdale tonight unhappy,1,"[there, isnt, a, new, riverdale, tonight, unha..."


In [18]:
all_words = [word for tokens in df['tokens'] for word in tokens] # Все слова во всех твитах
tweet_length = [len(tokens) for tokens in df['tokens']] # список с количеством слов в твите
different_words = sorted(list(set(all_words))) # список различных слов
print('Всего слов в твитах: ', len(all_words))
print('Длина максимального твита: ', max(tweet_length))
print('Длина минимального твита: ', min(tweet_length))
print('Различных слов во всех твитах: ', len(different_words))

Всего слов в твитах:  33208
Длина максимального твита:  30
Длина минимального твита:  0
Различных слов во всех твитах:  6378


### 1.4. Функции для трансформорации слова в векторы с использованием различных подходов:
- бинаризация, переделка в onehot vector
- подсчет слов
- TFIDF

Все методы векторизации предназначены для перевода слов в вектора (числа)

про MultiLabelBinarizer() написано [тут](https://ru.stackoverflow.com/questions/928443/%D0%9A%D0%B0%D0%BA-%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D1%82-sklearn-preprocessing-multilabelbinarizer), [тут](https://spec-zone.ru/scikit_learn/modules/generated/sklearn.preprocessing.multilabelbinarizer) и [тут](https://scikit-learn.ru/1-12-multiclass-and-multioutput-algorithms/#multiclass-classification). В целом данный класс преобразует столбец с токенами в One-Hot-Encoded



In [115]:
def binarize_tokens_and_split(df, tokens_column_name, target_column_name, test_size=0.2):
    """
    Преобразует матрицу с токенами в one-hot-векторы и разбиват на тренировочную и тестовую
    :param df: датафрейм с токенами и целевой переменной
    :param tokens_column_name: название столбца датафрейма с токенами
    :param target_column_name: название столбца датафрейма с целевой переменной
    :param test_size: размер тестовой выборки
    :return: X_train_bin, X_test_bin, y_train, y_test
    """
    X_train, X_test, y_train, y_test = train_test_split(df[tokens_column_name],
                                                        df[target_column_name],
                                                        stratify=df[target_column_name],
                                                        test_size=test_size,
                                                        random_state=42)
    
    mlb = MultiLabelBinarizer()
    mlb.fit(X_train)  # получает классы - один класс - одно слово, где слова не повторяются
    X_train_bin = mlb.transform(X_train) # преобразуем в onehot vector
    X_test_bin = mlb.transform(X_test)  # преобразуем в onehot vector
    return X_train_bin, X_test_bin, y_train, y_test

про CountVectorizer() можно ознакомится [тута](https://github.com/Yorko/mlcourse.ai/blob/main/jupyter_russian/tutorials/vectorizers_tutorial_mvsamsonov.ipynb), [тута](https://habr.com/ru/articles/702626/) и [тут](https://habr.com/ru/articles/205360/). 
Если коротко то данный класс преобразует твиты в матрицу значениями которой, являются количества вхождения данного ключа(слова) в текст. Следует отметить что здесь нужно передать не список токенов, а список предложений. По умолчанию возвращается разреженная матрица

In [116]:
def count_words_and_split(df, tokens_column_name, target_column_name, test_size=0.2):
    """
    Преобразует матрицу с токенами сначала в матрице предложений,
    Затем заносит в матрицу для каждого слова - сколько раз оно встречалось во всей выборке
    Возвращает разреженную матрицу
    
    :param df: датафрейм с токенами и целевой переменной
    :param tokens_column_name: название столбца датафрейма с токенами
    :param target_column_name: название столбца датафрейма с целевой переменной
    :param test_size: размер тестовой выборки
    :return: X_train_counts, X_test_counts, y_train, y_test
    """
    list_corpus = df[tokens_column_name].apply(lambda x: ' '.join(x)).tolist() # из токенов получаем как бы список твитов в одном предложении
    list_labels = df[target_column_name].tolist() # преобразуем целевые переменные в список
    
    X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, stratify=list_labels, test_size=test_size, random_state=21)
    
    count_vectorizer = CountVectorizer()
    X_train_counts = count_vectorizer.fit_transform(X_train).toarray()
    X_test_counts = count_vectorizer.transform(X_test).toarray()
    
    return X_train_counts, X_test_counts, y_train, y_test

    

про TFIDF написано [тут](https://github.com/Yorko/mlcourse.ai/blob/main/jupyter_russian/tutorials/vectorizers_tutorial_mvsamsonov.ipynb)
Если слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения CountVectorizer различий не будет. Здесь на помощь приходит TFIDF.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

In [72]:
def tfidf_and_split(df, tokens_column_name, target_column_name, test_size=0.2):
    """
    Преобазует матрицу с токенами в TFIDF
    
    :param df: датафрейм с токенами и целевой переменной
    :param tokens_column_name: название столбца датафрейма с токенами
    :param target_column_name: название столбца датафрейма с целевой переменной
    :param test_size: размер тестовой выборки
    :return: X_train_tfidf, X_test_tfidf, y_train, y_test
    """
    list_corpus = df[tokens_column_name].apply(lambda x: ' '.join(x)).tolist()
    list_labels = df[target_column_name].tolist()

    X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, stratify=list_labels, test_size=test_size, random_state=21)
    tfidf = TfidfVectorizer()
    tfidf.fit(X_train)
    X_train_tfidf = tfidf.transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)

    return X_train_tfidf, X_test_tfidf, y_train, y_test

### 1.5. Функции для препроцессинга слов перед векторизацией

In [73]:
def lemming(df, tokens_column_name):
    """
    Применяет лематизацию к токенам
    :param df: датафрейм
    :param tokens_column_name:  название столбца к которому нужно применить лематизацию
    :return: столбец с привденными леммами
    """
    lemmatizer = WordNetLemmatizer()
    return df[tokens_column_name].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [74]:
def stemming(df, tokens_column_name):
    """
    Применяет стемминг к токенам
    :param df: датафрейм
    :param tokens_column_name:  название столбца к которому нужно применить стемминг
    :return: столбец с привденными формами слов
    """
    stemmer = PorterStemmer()
    return df[tokens_column_name].apply(lambda x: [stemmer.stem(word) for word in x])

In [75]:
def stop_words_remove(df, tokens_column_name):
    """
    удаляет стоп слова
    :param df: датафрейм
    :param tokens_column_name:  название столбца в котором нужно удалить стоп слова
    :return: столбец с удаленными стоп словами
    """
    stop_words = set(stopwords.words('english'))
    return df[tokens_column_name].apply(lambda x: [word for word in x if word not in stop_words])

In [76]:
def misspelling(df, tokens_column_name):
    """
    Исправляет орфографические ошибки в словах
    :param df: датафрейм
    :param tokens_column_name:  название столбца в котором нужно исправить орфографические ошибки
    :return: столбец с исправлеными токенами
    """
    speller = Speller(lang="en")
    return df[tokens_column_name].apply(lambda words: [speller(word) for word in words])

### 1.6. Вспомогатльная функция для заполнения таблицы

In [77]:
def make_classification(prep_name, df, tokens_column, target_column, model):
    result = [prep_name]
    X_train, X_test, y_train, y_test = binarize_tokens_and_split(df, tokens_column, target_column)
    result.append(accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test)))

    X_train, X_test, y_train, y_test = count_words_and_split(df, tokens_column, target_column)
    result.append(accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test)))

    X_train, X_test, y_train, y_test = tfidf_and_split(df, tokens_column, target_column)
    result.append(accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test)))

    return result

## 2. ML-алгоритмы
Поскольк нам дали возможность выбирать модели самим, будем пробовать четыре модели:
- логистическая регрессия
- дерево решений
- случайный лес
- метод опорных векторов

Так же попробуем использовать GridSeach для выбора оптиального параметра для каждо модели.
Поскольку есть много вариаций для которой можно применить GreadSearch, мы сначала применим его для TFIDF с предварительным препроцессингом датасета (стеминг лематизация и удаления стоп слов)
Затем выявим лучшие параметры и уже модель с этими параметрами применим ко всем вариациям согласно заданию и заполним таблицы для каждой модели.

### 2.0. Для наглядности как работают лематизация, стемминг, исправление ошибок и удаление стоп слов - создадим новый датафрейм
Посмотрим как выглядит столбец с токенами после каждой из данных процедур

In [89]:
# сначала попробуем выбрать лучшие параметры получные для тренировочной выборки с использованием GridSearch
df_temp = df.copy()
df_temp.drop('tweet', axis=1, inplace=True)
df_temp['tokens_spell'] = misspelling(df_temp, 'tokens') # сначал справим ошибки в словах
df_temp['tokens_s_lemma'] = lemming(df_temp, 'tokens_spell') # потом применим лематизацию
df_temp['tokens_s_l_stem'] = stemming(df_temp, 'tokens_s_lemma') # потом стемминг
df_temp['tokens_total'] = stemming(df_temp, 'tokens_s_l_stem') # и затем удалим стоп-слова
df_temp

,target,tokens,tokens_spell,tokens_s_lemma,tokens_s_l_stem,tokens_total
0,1,"[how, unhappy, some, dogs, like, it, though]","[how, unhappy, some, dogs, like, it, though]","[how, unhappy, some, dog, like, it, though]","[how, unhappi, some, dog, like, it, though]","[how, unhappi, some, dog, like, it, though]"
1,1,"[talking, to, my, over, driver, about, where, im, goinghe, said, hed, love, to, go, to, new, york, too, but, since, trump, its, probably, not]","[talking, to, my, over, driver, about, where, im, going, said, hed, love, to, go, to, new, york, too, but, since, trump, its, probably, not]","[talking, to, my, over, driver, about, where, im, going, said, hed, love, to, go, to, new, york, too, but, since, trump, it, probably, not]","[talk, to, my, over, driver, about, where, im, go, said, hed, love, to, go, to, new, york, too, but, sinc, trump, it, probabl, not]","[talk, to, my, over, driver, about, where, im, go, said, hed, love, to, go, to, new, york, too, but, sinc, trump, it, probabl, not]"
2,1,"[does, anybody, know, if, the, rands, likely, to, fall, against, the, dollar, i, got, some, money, i, need, to, change, into, r, but, it, keeps, getting, stronger, unhappy]","[does, anybody, know, if, the, hands, likely, to, fall, against, the, dollar, i, got, some, money, i, need, to, change, into, r, but, it, keeps, getting, stronger, unhappy]","[doe, anybody, know, if, the, hand, likely, to, fall, against, the, dollar, i, got, some, money, i, need, to, change, into, r, but, it, keep, getting, stronger, unhappy]","[doe, anybodi, know, if, the, hand, like, to, fall, against, the, dollar, i, got, some, money, i, need, to, chang, into, r, but, it, keep, get, stronger, unhappi]","[doe, anybodi, know, if, the, hand, like, to, fall, against, the, dollar, i, got, some, money, i, need, to, chang, into, r, but, it, keep, get, stronger, unhappi]"
3,1,"[i, miss, going, to, gigs, in, liverpool, unhappy]","[i, miss, going, to, gigs, in, liverpool, unhappy]","[i, miss, going, to, gig, in, liverpool, unhappy]","[i, miss, go, to, gig, in, liverpool, unhappi]","[i, miss, go, to, gig, in, liverpool, unhappi]"
4,1,"[there, isnt, a, new, riverdale, tonight, unhappy]","[there, isnt, a, new, riverdale, tonight, unhappy]","[there, isnt, a, new, riverdale, tonight, unhappy]","[there, isnt, a, new, riverdal, tonight, unhappi]","[there, isnt, a, new, riverd, tonight, unhappi]"
...,...,...,...,...,...,...
3868,3,"[thanks, for, the, recent, follow, happy, to, connect, happy, have, a, great, thursday, get, this]","[thanks, for, the, recent, follow, happy, to, connect, happy, have, a, great, thursday, get, this]","[thanks, for, the, recent, follow, happy, to, connect, happy, have, a, great, thursday, get, this]","[thank, for, the, recent, follow, happi, to, connect, happi, have, a, great, thursday, get, thi]","[thank, for, the, recent, follow, happi, to, connect, happi, have, a, great, thursday, get, thi]"
3869,3,"[top, engaged, members, this, week, happy]","[top, engaged, members, this, week, happy]","[top, engaged, member, this, week, happy]","[top, engag, member, thi, week, happi]","[top, engag, member, thi, week, happi]"
3870,3,"[ngam, to, weeks, left, for, cadet, pilot, exam, crying, with, joy]","[nam, to, weeks, left, for, cadet, pilot, exam, crying, with, joy]","[nam, to, week, left, for, cadet, pilot, exam, cry, with, joy]","[nam, to, week, left, for, cadet, pilot, exam, cri, with, joy]","[nam, to, week, left, for, cadet, pilot, exam, cri, with, joy]"
3871,3,"[great, youre, welcome, josh, happy, adam]","[great, youre, welcome, josh, happy, adam]","[great, youre, welcome, josh, happy, adam]","[great, your, welcom, josh, happi, adam]","[great, your, welcom, josh, happi, adam]"


Затем преобразуем конечные токены в TFIDF и разобъем на тренировочный и тестовый датасеты

In [92]:
X_train_tf, X_test_tf, y_train, y_test = tfidf_and_split(df_temp, target_column_name='target', tokens_column_name='tokens_total')

### 2.0.1. Для начала предварительно выберем "наилучшие" параметры для каждой модели

In [112]:
# 1. ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ
clf = LogisticRegression()
param_grid = {'C': [0.1, 1, 10, 30, 50],
              'solver': ['newton-cg', 'sag', 'saga'],
              'penalty': ['l2', 'l1', 'elasticnet'],
              'max_iter': [5000],
              'random_state': [21],
              'class_weight': ['balanced'],
              'multi_class': ['multinomial'],
              'n_jobs': [-1]
              }
gs = GridSearchCV(clf, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'C': 10, 'class_weight': 'balanced', 'max_iter': 5000, 'multi_class': 'multinomial', 'n_jobs': -1, 'penalty': 'l2', 'random_state': 21, 'solver': 'saga'}
лучший score: 0.8837881077700767


In [109]:
# 2. ДЕРЕВО РЕШЕНИЙ
tree = DecisionTreeClassifier()
param_grid = {'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [21]
            }
gs = GridSearchCV(tree, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': None, 'criterion': 'gini', 'max_depth': 25, 'random_state': 21}
лучший score: 0.8618453280525301


In [110]:
# 3. Случайный лес
forest = RandomForestClassifier()
param_grid = {'n_estimators': [5, 10, 50, 100, 200, 300],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [21]
              }
 
gs = GridSearchCV(forest, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 44, 'n_estimators': 300, 'random_state': 21}
лучший score: 0.883148157798739


In [111]:
svc = SVC()
param_grid = {'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None]
              }
gs = GridSearchCV(svc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train_tf, y_train)
print(f'лучшие параметры: {gs.best_params_}')
print(f'лучший score: {gs.best_score_}')

лучшие параметры: {'C': 1, 'class_weight': None, 'gamma': 'scale', 'kernel': 'linear'}
лучший score: 0.8860498202094951


Теперь когда мы получили примерно оптимальные гипперпараметры моежм приступить к следующему
В следующем шаге мы будем использовать именно "лучшие" параметры для каждой модели

### 2.1. Логистическая регрессия

In [117]:
%%time

clf = LogisticRegression(C=10,
                         class_weight='balanced',
                         max_iter=5000,
                         solver='saga',
                         multi_class='multinomial',
                         n_jobs=-1,
                         random_state=21,
                         penalty='l2',)

# Создаем датафрейм кудла будем заносить метрики
results_logreg = pd.DataFrame({'preprocces':[], '0 or 1 if the word exists': [], 'word counts':[], 'TFIDF':[]})

# Заполняем табличку
results_logreg.loc[len(results_logreg)] = make_classification('just tok', df, 'tokens', 'target', clf)
print("1 из 6. Расчет точности без препроцессинга завершен")

df['stemmed'] = stemming(df, 'tokens')
results_logreg.loc[len(results_logreg)] = make_classification('stemming', df, 'stemmed', 'target', clf)
print("2 из 6. Расчет точности после стемминга завершен")

df['lemmed'] = lemming(df, 'tokens')
results_logreg.loc[len(results_logreg)] = make_classification('lemming', df, 'lemmed', 'target', clf)
print("3 из 6. Расчет точности после лематизации завершен")

df['misspelled'] = misspelling(df, 'tokens')
df['misspelled+stemmed'] = stemming(df, 'misspelled')
results_logreg.loc[len(results_logreg)] = make_classification('misspelled+stemmed', df, 'misspelled+stemmed', 'target', clf)
print("4 из 6. Расчет точности после исправления орфографических ошибок и стеминга завершен")

df['misspelled'] = misspelling(df, 'tokens')
df['misspelled+lemmed'] = lemming(df, 'misspelled')
results_logreg.loc[len(results_logreg)] = make_classification('misspelled+lemmed', df, 'misspelled+lemmed', 'target', clf)
print("5 из 6. Расчет точности после исправления орфографических ошибок и лематизации завершен")

df['stop_words_removed'] = stop_words_remove(df, 'tokens')
results_logreg.loc[len(results_logreg)] = make_classification('stop_words_remove', df, 'stop_words_removed', 'target', clf)
print("6 из 6. Расчет точности после удаления стоп-слов завершен")

1 из 6. Расчет точности без препроцессинга завершен
2 из 6. Расчет точности после стемминга завершен
3 из 6. Расчет точности после лематизации завершен
4 из 6. Расчет точности после исправления орфографических ошибок и стеминга завершен
5 из 6. Расчет точности после исправления орфографических ошибок и лематизации завершен
6 из 6. Расчет точности после удаления стоп-слов завершен
CPU times: user 10min 12s, sys: 11.9 s, total: 10min 23s
Wall time: 10min 8s


In [118]:
results_logreg

,preprocces,0 or 1 if the word exists,word counts,TFIDF
0,just tok,0.908387,0.885161,0.887742
1,stemming,0.910968,0.889032,0.894194
2,lemming,0.908387,0.885161,0.891613
3,misspelled+stemmed,0.912258,0.883871,0.896774
4,misspelled+lemmed,0.909677,0.883871,0.896774
5,stop_words_remove,0.889032,0.890323,0.894194


### 2.2. Дерево решений

In [119]:
%%time

tree = DecisionTreeClassifier(class_weight=None,
                              criterion='gini',
                              max_depth=25,
                              random_state=21)

# Создаем датафрейм кудла будем заносить метрики
results_tree = pd.DataFrame({'preprocces':[], '0 or 1 if the word exists': [], 'word counts':[], 'TFIDF':[]})

# Заполняем табличку
results_tree.loc[len(results_tree)] = make_classification('just tok', df, 'tokens', 'target', tree)
print("1 из 6. Расчет точности без препроцессинга завершен")

df['stemmed'] = stemming(df, 'tokens')
results_tree.loc[len(results_tree)] = make_classification('stemming', df, 'stemmed', 'target', tree)
print("2 из 6. Расчет точности после стемминга завершен")

df['lemmed'] = lemming(df, 'tokens')
results_tree.loc[len(results_tree)] = make_classification('lemming', df, 'lemmed', 'target', tree)
print("3 из 6. Расчет точности после лематизации завершен")

df['misspelled'] = misspelling(df, 'tokens')
df['misspelled+stemmed'] = stemming(df, 'misspelled')
results_tree.loc[len(results_tree)] = make_classification('misspelled+stemmed', df, 'misspelled+stemmed', 'target', tree)
print("4 из 6. Расчет точности после исправления орфографических ошибок и стеминга завершен")

df['misspelled'] = misspelling(df, 'tokens')
df['misspelled+lemmed'] = lemming(df, 'misspelled')
results_tree.loc[len(results_tree)] = make_classification('misspelled+lemmed', df, 'misspelled+lemmed', 'target', tree)
print("5 из 6. Расчет точности после исправления орфографических ошибок и лематизации завершен")

df['stop_words_removed'] = stop_words_remove(df, 'tokens')
results_tree.loc[len(results_tree)] = make_classification('stop_words_remove', df, 'stop_words_removed', 'target', tree)
print("6 из 6. Расчет точности после удаления стоп-слов завершен")

1 из 6. Расчет точности без препроцессинга завершен
2 из 6. Расчет точности после стемминга завершен
3 из 6. Расчет точности после лематизации завершен
4 из 6. Расчет точности после исправления орфографических ошибок и стеминга завершен
5 из 6. Расчет точности после исправления орфографических ошибок и лематизации завершен
6 из 6. Расчет точности после удаления стоп-слов завершен
CPU times: user 1min 24s, sys: 338 ms, total: 1min 24s
Wall time: 1min 24s


In [120]:
results_tree

,preprocces,0 or 1 if the word exists,word counts,TFIDF
0,just tok,0.898065,0.878710,0.868387
1,stemming,0.886452,0.873548,0.867097
2,lemming,0.895484,0.876129,0.861935
3,misspelled+stemmed,0.891613,0.863226,0.876129
4,misspelled+lemmed,0.895484,0.876129,0.876129
5,stop_words_remove,0.869677,0.869677,0.868387
